In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar10

In [ ]:
(X_train , y_train) , (X_test , y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [ ]:
X_train = X_train.astype(float) / 255.0
X_test = X_test.astype(float) / 255.0

In [ ]:
from keras import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization

In [ ]:
model = Sequential()
model.add(Conv2D(16 , (3 , 3), activation='relu',input_shape = (32 , 32 , 3), kernel_initializer='he_uniform', padding='same' , kernel_regularizer='l2'))
model.add(Conv2D(16 , (3 , 3), activation='relu', kernel_initializer='he_uniform', padding='same' , kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(MaxPool2D(2 , 2))
model.add(Dropout(0.1))
model.add(Conv2D(32 , (5 , 5), activation='relu', kernel_initializer='he_uniform', padding='same' , kernel_regularizer='l2'))
model.add(Conv2D(32 , (5 , 5), activation='relu', kernel_initializer='he_uniform', padding='same' , kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(MaxPool2D(2 , 2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(32 , activation='relu', kernel_initializer='he_uniform', kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(10 , activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy ', metrics=['accuracy'])

In [ ]:
y_train1 = keras.utils.to_categorical(y_train)
y_test1 = keras.utils.to_categorical(y_test)

In [ ]:
history = model.fit(X_train , y_train1 , epochs = 3 , batch_size = 32 , validation_split=0.2 , use_multiprocessing=True)

Epoch 1/3
1250/1250 [==============================] - 175s 140ms/step - loss: 2.7937 - accuracy: 0.4433 - val_loss: 1.8705 - val_accuracy: 0.5500
Epoch 2/3
1250/1250 [==============================] - 177s 141ms/step - loss: 1.7055 - accuracy: 0.5833 - val_loss: 1.6372 - val_accuracy: 0.5688
Epoch 3/3
1250/1250 [==============================] - 171s 137ms/step - loss: 1.5367 - accuracy: 0.6203 - val_loss: 1.6964 - val_accuracy: 0.5746


In [ ]:
y_train1[:1000].size

10000

In [ ]:
model.evaluate(X_test , y_test1 , verbose = 0)

[1.5697920322418213, 0.5485000014305115]

In [ ]:
model.evaluate(X_test , y_test1 , verbose = 0)

[2.3026092052459717, 0.10000000149011612]

In [ ]:
model.evaluate(X_test , y_test1 , verbose = 0)

[1.7099330425262451, 0.5715000033378601]

In [ ]:
import tempfile

In [ ]:
_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

Saving model to:  /tmp/tmp4q_ldahj.h5


In [ ]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))

Size of gzipped baseline Keras model: 398103.00 bytes


In [ ]:
pip install tensorflow_model_optimization

     |████████████████████████████████| 174kB 7.9MB/s 


In [ ]:
import tensorflow_model_optimization as tfmot

In [ ]:
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization
clustering_params = {
  'number_of_clusters': 32,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}

In [ ]:
clustered_model = cluster_weights(model, **clustering_params)

In [ ]:
_, keras_file1 = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file1)
tf.keras.models.save_model(clustered_model, keras_file1, include_optimizer=False)

Saving model to:  /tmp/tmpjatd4f0y.h5


In [ ]:
print("Size of gzipped model after wieght clustering: %.2f bytes" % (get_gzipped_model_size(keras_file1)))

Size of gzipped model after wieght clustering: 102984.00 bytes


In [ ]:
batch_size = 64
epochs = 2
validation_split = 0.175
import numpy as np

In [ ]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
num_images = 32 * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

In [ ]:
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.1,
                                                               final_sparsity=0.50,
                                                               begin_step=0,
                                                               end_step=end_step)
}

In [ ]:
model_for_pruning = prune_low_magnitude(model, **pruning_params)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [ ]:
_, keras_file2 = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file2)
tf.keras.models.save_model(model_for_pruning, keras_file2, include_optimizer=False)

Saving model to:  /tmp/tmp_1izwzqz.h5


In [ ]:
print("Size of gzipped model after pruning: %.2f bytes" % (get_gzipped_model_size(keras_file2)))

Size of gzipped model after pruning: 401977.00 bytes


In [ ]:
clustered_model = cluster_weights(model_for_pruning, **clustering_params)

ValueError: ignored

In [ ]:
model = Sequential([
cluster_weights(Conv2D(16 , (3 , 3), activation='relu',input_shape = (32 , 32 , 3), kernel_initializer='he_uniform', padding='same' , kernel_regularizer='l2') , **clustering_params),
cluster_weights(Conv2D(16 , (3 , 3), activation='relu',input_shape = (32 , 32 , 3), kernel_initializer='he_uniform', padding='same' , kernel_regularizer='l2') , **clustering_params),
BatchNormalization(),
MaxPool2D(2 , 2),
Dropout(0.1),
cluster_weights(Conv2D(32 , (5 , 5), activation='relu', kernel_initializer='he_uniform', padding='same' , kernel_regularizer='l2' ), **clustering_params),
cluster_weights(Conv2D(32 , (5 , 5), activation='relu', kernel_initializer='he_uniform', padding='same' , kernel_regularizer='l2' ), **clustering_params),
BatchNormalization(),
MaxPool2D(2 , 2),
Dropout(0.2),
Flatten(),
prune_low_magnitude(Dense(32 , activation='relu', kernel_initializer='he_uniform', kernel_regularizer='l2' ) , **pruning_params),
BatchNormalization(),
Dropout(0.3),
Dense(10 , activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy ', metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [ ]:
_, keras_file4 = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file4)
tf.keras.models.save_model(model, keras_file4, include_optimizer=False)

Saving model to:  /tmp/tmp7z1k2hjo.h5


In [ ]:
print("Size of gzipped model after pruning: %.2f bytes" % (get_gzipped_model_size(keras_file4)))

Size of gzipped model after pruning: 103593.00 bytes


In [ ]:
_, keras_file4 = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file4)
tf.keras.models.save_model(model, keras_file4, include_optimizer=False)

Saving model to:  /tmp/tmp8tnlvyi6.h5


In [ ]:
print("Size of gzipped model after pruning: %.2f bytes" % (get_gzipped_model_size(keras_file4)))

Size of gzipped model after pruning: 288877.00 bytes


In [ ]:
quantize_model = tfmot.quantization.keras.quantize_model

In [ ]:
tfmot.quantization.keras.quantize_scope(
    *pruning_params,
    *clustering_params
)

In [ ]:
q_aware_model = quantize_model(model)

ValueError: ignored